In [3]:
# Required imports
import numpy as np
import pickle 

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix, classification_report

### Loading Data & looking around what it contains (data exploration)

In [54]:
#Get sentences info from sentences.pkl
with open('D:/DataScience/D2FRecrutement/NLP/Data/sentences.pkl', 'rb') as f:
    data_sentences = pickle.load(f)
len(data_sentences)
#data_sentences

45930

In [55]:
data_sentences = data_sentences[34447:]

In [13]:
#Get sequences info from sequences.pkl
with open('D:/DataScience/D2FRecrutement/NLP/Data/sequences.pkl', 'rb') as f:
    data_sequences = pickle.load(f)
len(data_sequences)
#data_sequences

45930

In [14]:
#Get labels info from labels.pkl
with open('D:/DataScience/D2FRecrutement/NLP/Data/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
len(labels)
#labels

45930

In [56]:
labels = labels[34447:]

In [26]:
#Get dictionary info from dict.pkl
with open('D:/DataScience/D2FRecrutement/NLP/Data/dict.pkl', 'rb') as f:
    dictionary = pickle.load(f)
len(dictionary)
#dictionary

30432

In [57]:
data_sentences[3]

"Les sous-officiers ont toujours été l'âme d'une armée."

In [58]:
data_sequences[35]

[104, 311, 0, 38, 189, 1540, 30057, 8, 8645, 1, 5, 2707, 586, 24, 2849, 2]

In [59]:
dicionary

{',': 0,
 'de': 1,
 '.': 2,
 'la': 3,
 'et': 4,
 "l'": 5,
 'à': 6,
 'le': 7,
 'les': 8,
 'des': 9,
 "d'": 10,
 'est': 11,
 'en': 12,
 'que': 13,
 'qui': 14,
 'un': 15,
 'une': 16,
 'pour': 17,
 'dans': 18,
 'du': 19,
 'je': 20,
 'il': 21,
 'nous': 22,
 'vous': 23,
 '-': 24,
 'au': 25,
 'a': 26,
 'ce': 27,
 "c'": 28,
 'plus': 29,
 "qu'": 30,
 'pas': 31,
 'par': 32,
 'notre': 33,
 "s'": 34,
 'sur': 35,
 'ne': 36,
 'france': 37,
 'nos': 38,
 'avec': 39,
 'cette': 40,
 'se': 41,
 'mais': 42,
 'pays': 43,
 'sont': 44,
 'aussi': 45,
 'elle': 46,
 'aux': 47,
 "n'": 48,
 'ont': 49,
 'être': 50,
 'leur': 51,
 'tout': 52,
 'votre': 53,
 "j'": 54,
 'tous': 55,
 'son': 56,
 'y': 57,
 '"': 58,
 'on': 59,
 'bien': 60,
 'ces': 61,
 'même': 62,
 'ses': 63,
 'ou': 64,
 'comme': 65,
 'entre': 66,
 ':': 67,
 'europe': 68,
 'sa': 69,
 "aujourd'hui": 70,
 'monde': 71,
 'doit': 72,
 'faire': 73,
 'français': 74,
 'ai': 75,
 'ils': 76,
 'si': 77,
 'faut': 78,
 'sans': 79,
 'été': 80,
 'fait': 81,
 'président

### Changing labels from ("C","M") to (0,1)

In [60]:
def change_labels(labels):
    y = [None]*len(labels)
    for x in range(len(labels)):
        if labels[x] == 'C':
            y[x] = 0
        else:
            y[x] = 1
            
    return y
            

In [61]:
New_labels = change_labels(labels)

### Make TF-IDF

In [62]:
vect_tf_idf = TfidfVectorizer(min_df = 20, lowercase = True, use_idf = True).fit(data_sentences)

### Split given data to train and test to evaluate the model
### before apply it to the required test sentences

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(data_sentences, New_labels, random_state = 0)

### Transform Train data to TF-IDF vector

In [64]:
x_train = vect_tf_idf.transform(X_train)

### Loading Deep Learning packages

In [65]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils.np_utils import to_categorical

### Convert labels to categorical format

In [67]:
y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

### Deep Learning Model

In [68]:
model = Sequential()
model.add(Dense(units=512, kernel_initializer='normal', activation='selu', input_dim=1418))
model.add(Dropout(0.20))
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, kernel_initializer='normal', activation='softmax', input_dim=16))
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

### Train the Model

In [69]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
8612/8612 [==============================] - 10s 1ms/step - loss: 0.5756 - acc: 0.6908
Epoch 2/5
8612/8612 [==============================] - 8s 935us/step - loss: 0.4501 - acc: 0.7880
Epoch 3/5
8612/8612 [==============================] - 7s 764us/step - loss: 0.4058 - acc: 0.8098
Epoch 4/5
8612/8612 [==============================] - 7s 779us/step - loss: 0.3751 - acc: 0.8208
Epoch 5/5
8612/8612 [==============================] - 7s 777us/step - loss: 0.3469 - acc: 0.8326


### Evaluate the Model

In [70]:
loss_and_metrics = model.evaluate(vect_tf_idf.transform(X_test), y_test, batch_size=128)

2871/2871 [==============================] - 0s 133us/step


In [71]:
loss_and_metrics

[0.6778688230027891, 0.7345872522438498]

### So here we built model with 73.4% accuracy

### Playing around Data to see the result
### hERE 0 indicates Chirac & 1 for Mitterrand) 

In [110]:
New_labels[7]

0

In [158]:
data_sentences[7]

"C'est dans ce contexte que s'inscrit la construction de votre bibliothèque municipale à vocation régionale, dont nous avons admiré tout à l'heure la maquette, bibliothèque Georges-POMPIDOU, en faveur de laquelle, vous avez fait un travail considérable et, je n'en doute pas, une réussite exceptionnelle."

In [113]:
New_labels[7777]

1

In [159]:
data_sentences[7777]

'Notre ministre des affaires étrangères, M. Roland Dumas, ici présent, est revenu à Prague directement de Moscou.'

In [114]:
New_labels[199]

0

In [167]:
data_sentences[199]

'Ce délai est nécessaire pour parvenir à des solutions adaptées aux mutuelles relevant du code de la mutualité et prenant en compte, pour tous les acteurs concernés, les particularités du secteur de la santé.'

In [166]:
New_labels[6001]

1

In [168]:
data_sentences[6001]

"L'Europe - cela lui arrive tous les 5 ou 6 ans - s'arrête, s'interroge, hésite sur son destin."